# Complete pipeline

## Introductory notes:
This notebook presents minimal functionality needed to go through the cleaning, ICA, spectral and event analysis stages.
* For the cleaning part, the functionality consists of resampling, filtering, bad channels and bad data spans annotation and bad channels interpolation.
* For the ICA part, it is fitting and selecting components you want to exclude.
* For the spectral analyses part, it is spectrogram+hypnogram, PSD per sleep stage and topomap per sleep stage per frequency band.
* For the events detection, it is spindles, slow waves and rapid eye movements detection and analysis.

For the extended functionality check out the corresponding notebooks.

## Import module

In [ ]:
import os
import sys
sys.path.append(os.path.abspath('..'))

In [ ]:
import pooch
from sleepeegpy.pipeline import (
    CleaningPipe, ICAPipe, SpectralPipe, 
    SpindlesPipe, SlowWavesPipe, RapidEyeMovementsPipe
    )
from sleepeegpy.dashboard import create_dashboard

## Setup Input Files

By default, all the input files are assumed to be saved in <b>input_files</b>, which will be created (if not already exists) in the notebook path. Change the following strings to use another path.
Changing the output directory is also optional.

In [ ]:
from os import makedirs

output_dir  = "output_folder" # Output path and name can be changed here
input_dir = "input_files" # input files dir can be changed here
makedirs(input_dir, exist_ok=True)
makedirs(output_dir, exist_ok=True)

#### Add required files
* Put all your files in the input folder.
* Modify your eeg file name below. The file can be any format supported by the mne.read_raw() function.
* Modify your hypnogram file name (Point-per-row type of hypnogram) below.
* If needed, change Hypnogram's sampling frequency 
* For more information about the supported formats, see [mne documentation](https://mne.tools/stable/generated/mne.io.Raw.html)

In [ ]:
eeg_file_name= "resampled_raw.fif" # add your eeg_path here
hypnogram_filename = "staging.txt" # Point-per-row type of hypnogram.
hypno_freq = 1 # If required, change Hypnogram's sampling frequency (visbrain's hypnograms default to 1)

#### Adjust variables
* Add your subject code in the following code.
* If required, change n_components - should be equal or less then number of channels. see [more information](https://mne.tools/stable/generated/mne.preprocessing.ICA.html)
* Adjust spectrogram electrode, loc_chname and roc_chname. Make sure they fit the channel names in the montage.

In [ ]:
subject_code =  "AA41"
n_components = 30
spectrogram_electrode = 'E101'
loc_chname = "E252"
roc_chname = "E226"

## Retrieve the example dataset

In [ ]:
hypnogram_path = os.path.join(input_dir,hypnogram_filename)
cache_dir = pooch.os_cache("sleepeegpy_dataset")
doi = "10.5281/zenodo.10362189"
odie = pooch.create(
    path=cache_dir,
    base_url=f"doi:{doi}",
)
odie.load_registry_from_doi()

In [ ]:
bad_channels = odie.fetch("bad_channels.txt")
annotations = odie.fetch("annotations.txt")
path_to_eeg = odie.fetch("resampled_raw.fif", progressbar=True)
for i in range(1,4):
    odie.fetch(f"resampled_raw-{i}.fif", progressbar=True)

## Cleaning

Initialize `CleaningPipe` object by providing it with path to eeg file and output directory in which you want the data to be saved.

In [ ]:
pipe = CleaningPipe(
    path_to_eeg=path_to_eeg,
    output_dir=output_dir,
)

### Resampling
This can take more than an hour depending on eeg signal size and specs of the computer you're running the analysis on.

In [ ]:
pipe.resample(sfreq=250)

In [ ]:
pipe.filter(l_freq=0.75, h_freq=40)

In [ ]:
pipe.notch(freqs="50s")

### Select bad channels and epochs

Select bad channels in the opened browser.

In [ ]:
pipe.plot(save_bad_channels=True)

In [ ]:
pipe.read_bad_channels()

In [ ]:
pipe.interpolate_bads(reset_bads=True)

Select bad epochs

Click "a" -> "Add description" -> Enter BAD_EPOCH -> Annotate bad data spans

In [ ]:
pipe.plot(butterfly=True, save_annotations=True,overwrite=True)

In [ ]:
pipe.read_annotations()

In [ ]:
fig = create_dashboard(
    subject_code=subject_code, 
    prec_pipe=pipe, 
    hypno_psd_pick=spectrogram_electrode,
    hypnogram=hypnogram_path,
    hypno_freq=hypno_freq,
    reference="average")

## ICA

Pass the preceding (cleaning) pipe to the ICAPipe.

In [ ]:
ica_pipe = ICAPipe(prec_pipe=pipe, n_components=n_components)

Fit the ICA on the 1 Hz high-pass filtered data.

In [ ]:
ica_pipe.fit()

Visually inspect ICA components.

In [ ]:
ica_pipe.plot_sources()

Pass to the `exclude` argument indices of components you want to remove from the raw signal.

In [ ]:
ica_pipe.apply()

## Spectral

Pass the preceding (cleaning or ICA) pipe to the SpectralPipe. Also provide pass to the hypnogram and don't forget to pass its frequency to the corresponding parameter.

In [ ]:
spectral_pipe = SpectralPipe(
    prec_pipe=ica_pipe,
    path_to_hypno=hypnogram_path,
    hypno_freq=hypno_freq,
)

If you don't have a hypnogram, you can use the method `predict_hypno`, which will use [YASA's algorithm](https://raphaelvallat.com/yasa/build/html/generated/yasa.SleepStaging.html#yasa.SleepStaging). 

Make sure that the electrodes you provide are clean.

In [ ]:
# spectral_pipe.predict_hypno(
#     eeg_name = "E183",
#     eog_name = "E252",
#     emg_name = "E247",
#     ref_name = "E26",
#     save=False
# )

Pass an electrode name to calculate spectrogram for (e.g., E101)

In [ ]:
spectral_pipe.plot_hypnospectrogram(picks=[spectrogram_electrode])

In [ ]:
spectral_pipe.compute_psd(
    sleep_stages={"Wake": 0, "N1": 1, "N2/3": (2, 3), "REM": 4},
    reference="average",
    # Additional arguments passed to the Welch method:
    n_fft=1024,
    n_per_seg=1024,
    n_overlap=512,
    window="hamming",
    verbose=False
)

In [ ]:
spectral_pipe.plot_psds(picks=[spectrogram_electrode], psd_range=(-30, 30))

Create a collage with rows for sleep stages and columns for bands.

In [ ]:
spectral_pipe.plot_topomap_collage()

## Events

Pass the preceding (cleaning or ICA or spectral) pipe to one of the SpindlesPipe, SlowWavesPipe or RapidEyeMovementsPipe. If the preceding is cleaning or ICA - provide path to the hypnogram and don't forget to pass its frequency to the corresponding parameter.

In [ ]:
spindles_pipe = SpindlesPipe(prec_pipe=spectral_pipe)

spindles_pipe.detect()
spindles_pipe.plot_average(
    center="Peak",
    hue="Stage",
    time_before=1,
    time_after=1,
)

In [ ]:
spindles_pipe.results.summary(grp_chan=False, grp_stage=True)

In [ ]:
spindles_pipe.compute_tfr(freqs=(10, 20), n_freqs=100, time_before=1, time_after=1)
spindles_pipe.tfrs["N2"].plot([spectrogram_electrode])

In [ ]:
slow_waves_pipe = SlowWavesPipe(prec_pipe=spindles_pipe)

slow_waves_pipe.detect()
slow_waves_pipe.plot_average(
    center="NegPeak",
    hue="Stage",
    time_before=0.4,
    time_after=0.8,
)

In [ ]:
slow_waves_pipe.compute_tfr(
    freqs=(0.5, 5), n_freqs=100, time_before=4, time_after=4, n_cycles=2
)
slow_waves_pipe.tfrs["N3"].plot([spectrogram_electrode])

In [ ]:
rems_pipe = RapidEyeMovementsPipe(prec_pipe=slow_waves_pipe)

rems_pipe.detect(
    loc_chname=loc_chname,
    roc_chname=roc_chname,
)

rems_pipe.plot_average(
    center="Peak",
    time_before=0.5,
    time_after=0.5,
    filt=(None, None),
    mask=None,
)